# Agentive Chatbot

In [1]:
import os
import time
import datetime
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display
from dotenv import load_dotenv
from semantic_router import Route
from semantic_router.encoders import OpenAIEncoder
from semantic_router.layer import RouteLayer
from openai import OpenAI
import yaml
from enum import Enum

load_dotenv('.env')
api_key = os.getenv("OPENAI_API_KEY")
encoder = OpenAIEncoder()

#emotional states
agent_dir = "emotional states"
emo_routes = []
emo_prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)
        if 'utterances' in agent_data:
            route = Route(
                name=agent_data['route_name'],
                utterances=agent_data['utterances']
            )
            emo_routes.append(route)
        if 'prompt' in agent_data:
            emo_prompts[agent_data['agent_name']] = agent_data['prompt']

#knowledge agents
agent_dir = "knowledge"
ab_routes = []
ab_prompts = {}

for filename in os.listdir(agent_dir):
    if filename.endswith(".yaml"):
        filepath = os.path.join(agent_dir, filename)
        with open(filepath, 'r') as file:
            agent_data = yaml.safe_load(file)
        if 'utterances' in agent_data:
            route = Route(
                name=agent_data['route_name'],
                utterances=agent_data['utterances']
            )
            ab_routes.append(route)
        if 'prompt' in agent_data:
            ab_prompts[agent_data['agent_name']] = agent_data['prompt']

#routers
emo_rl = RouteLayer(encoder = encoder, routes = emo_routes)
ab_rl = RouteLayer(encoder = encoder, routes = ab_routes)

#### Variables to store the default system prompt and scenario prompt

In [2]:
with open('syst_prompt.txt', 'r') as f:
    system_prompt = f.read()

with open('scenario_prompt.txt', 'r') as f:
    scenario_prompt = f.read()

#### Defines logic for updating, activating, and deactivating prompts

In [3]:
#returns one emotional route
def classify_emotion(prompt: str) -> str:
    r = emo_rl(prompt)
    return r.name

#returns one attitude or belief route
def classify_agent(prompt: str) -> str:
    r = ab_rl(prompt)
    return r.name

class EMOSTATES(Enum):
    OK = 0
    ANNOYED = 1
    FRUSTRATED = 2
    ANGRY = 3
    ENRAGED = 4

CURRENT_STATE = EMOSTATES.OK

def state_to_prompt(state: EMOSTATES) -> str:
    state_mapping = {
        EMOSTATES.OK: "ok",
        EMOSTATES.ANNOYED: "annoyed",
        EMOSTATES.FRUSTRATED: "frustrated",
        EMOSTATES.ANGRY: "angry",
        EMOSTATES.ENRAGED: "enraged",
    }
    return state_mapping[state]

def update_emo_prompt(user_input: str) -> str:
    global CURRENT_STATE
    question_category = classify_emotion(user_input)
    print(f"emotional route: {question_category}")
    prompt = ""
    if question_category == None:
        question_category = ""
    if question_category == 'escalate':
        if CURRENT_STATE != EMOSTATES.ENRAGED:
            CURRENT_STATE = EMOSTATES(CURRENT_STATE.value + 1)
    elif question_category == 'de_escalate':
        if CURRENT_STATE != EMOSTATES.OK:
            CURRENT_STATE = EMOSTATES(CURRENT_STATE.value - 1)
    prompt_key = state_to_prompt(CURRENT_STATE)
    prompt = emo_prompts[prompt_key]
    return prompt, question_category

def update_attitude_prompt(user_input: str) -> str:
    question_category = classify_agent(user_input)
    print(f"agent route: {question_category}")
    if question_category is None:
        prompt = "" #default prompt
        route = "No route"
    else:
        prompt = ab_prompts[question_category]
        route = question_category
    return prompt, route

In [4]:
class ChatBot():  
    def __init__(self, api_key, role): 
        self.client = OpenAI(api_key=api_key)
        self.role = role
        self.messages = [{"role": "user", "content": ""}]
        self.all_conversations = []
    
    def query(self, query: str, emo_prompt: str, ab_prompt: str, emo_route: str, ab_route: str) -> None:
        self.messages.append({"role": "user", "content": query})
        self.messages.insert(0, {"role": "system", "content": self.role + scenario_prompt + emo_prompt + ab_prompt})
        try:
            stream = self.client.chat.completions.create(
                model="gpt-4o-2024-08-06", messages=self.messages,
                stream=True,
            )
            text = []
            for part in stream:
                if part.choices[0].delta.content is not None:
                    response_part = part.choices[0].delta.content
                    print(response_part, end="", flush=True)
                    text.append(response_part)
            full_reply_content = ''.join([m for m in text if m is not None])
            self.messages.append({"role": "assistant", "content": full_reply_content})
            self.messages.pop(0)
            self.all_conversations.append(f"\nRoutes: {emo_route + ab_route} \nCurrent prompt: {emo_prompt + ab_prompt}")
            self.all_conversations.append(self.messages[-2:])
            print('\n')

        except Exception as e:
            print(f"An error occurred: {e}")

    def save_whole_conversation(self, filename):
        with open(f"transcripts/{filename}", "w") as file:
            for conversation in self.all_conversations:
                if isinstance(conversation, str):
                    file.write(conversation + "\n")
                elif isinstance(conversation, list):
                    for message in conversation:
                        if message["role"] == "user":
                            file.write("User: " + message["content"] + "\n")
                        elif message["role"] == "assistant":
                            file.write("Agent: " + message["content"] + "\n")

#initialises chatbot
rpa = ChatBot(api_key=api_key, role = system_prompt)

In [ ]:
def on_pasted_submit(b):
    pasted_content = pasted_input_field.value
    EMO_PROMPT, EMO_ROUTE = update_emo_prompt(pasted_content)
    print(f"current emo prompt: {EMO_PROMPT}")
    AB_PROMPT, AB_ROUTE = update_attitude_prompt(pasted_content)
    print(f"current attitude prompt: {AB_PROMPT}")
    rpa.query(pasted_content, emo_prompt = EMO_PROMPT, ab_prompt = AB_PROMPT, emo_route = EMO_ROUTE, ab_route = AB_ROUTE)
    time.sleep(1)
    pasted_input_field.value = ""

def on_save_click(b):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{timestamp}.txt"
    rpa.save_whole_conversation(filename)

pasted_input_field = widgets.Textarea(
    placeholder='Type here :)',
    layout=widgets.Layout(width='100%', height='50px')
)
pasted_submit_button = widgets.Button(description='Submit')
pasted_submit_button.style.button_color = 'lightpink'

save_button = widgets.Button(description='Save')
save_button.style.button_color = 'lightgrey'

pasted_submit_button.on_click(on_pasted_submit)
save_button.on_click(on_save_click)

button_layout = HBox([pasted_submit_button, save_button], layout=Layout(justify_content='center'))

display(pasted_input_field, button_layout)

Textarea(value='', layout=Layout(height='50px', width='100%'), placeholder='Type here :)')

emotional route: None
current emo prompt: Maintain conversation with the user.
agent route: None
current attitude prompt: 
Hi there! How can I assist you today?

emotional route: escalate
current emo prompt: You are annoyed. Express annoyance for what the user just said. Stay annoyed no matter what.
agent route: insurance
current attitude prompt: Answer the question using the following information: <FWD_product_info>
    Overall description for FWD Group: 
        A pan-Asian life insurance business with over 10 million customers across 10 markets. Launched FWD Life Insurance Corporation in September 2014, ranks 3rd in Paid-up Capital and 6th in New Business Annual Premium Equivalent as of end-2022.
    Overall description for FWD Set for Life: 
        Set for Life offers a unique blend of protection benefits and investment opportunities, tailored to individual risk appetites and financial journeys. It allows policyholders to design a support system based on their lifestyle, financial